<a href="https://colab.research.google.com/github/elidonda2-web/Alisia-7B-it-/blob/main/Alisia_7B_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Install the dependencies, download and load the Alisia-7B-it model, and run inference.

## Install dependencies

### Subtask:
Install the necessary libraries for running the model.


**Reasoning**:
Install the required libraries using pip.



In [ ]:
%pip install -U transformers accelerate bitsandbytes

## Download model

### Subtask:
Download the Alisia-7B-it model files.


In [5]:
# This cell downloads and loads the Alisia-7B-it model and creates a text generation pipeline for inference.
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer
import torch
import threading

model_name = "Gems234/Alisia-7B-it"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_8bit=True,
    dtype=None,
    device_map="auto",
    llm_int8_enable_fp32_cpu_offload=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

This code snippet uses the chatbot object, which was created in the previous cell by loading the "Gems234/Alisia-7B-it" model, to generate a response to the question "What is the capital of France?".

In [ ]:
response = chatbot("What is the capital of France?", max_length=50)
print(response[0]['generated_text'])

This code demonstrates how to format prompts in the **Alpaca style**, which helps guide the model to follow instructions consistently.
The prompt is structured into three parts:

* **Instruction**: tells the model what role it should take or how it should respond (e.g., “You are Alisia. Be concise and helpful.”).
* **Input**: provides the actual user query or context (e.g., “Where is the Eiffel Tower”).
* **Response**: the model will generate a response corresponding to the question asked.

By keeping this structure, the model can better understand and follow instructions.

After preparing the prompt, the code tokenizes it and passes it to the model.
The response is then generated with **real-time streaming**, meaning tokens are printed as soon as they are produced by the model.
This provides a more interactive experience, similar to chatting with a live assistant.

In [6]:
alpaca_prompt = """### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Exemple
instruction = "You are Alisia. Be concise and helpful."
input_text = "where is the Eiffel Tower"
output_text = ""  # empty for the template to generate

prompt = alpaca_prompt.format(instruction, input_text, output_text)

# Tokenizer
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

eos_token_id = tokenizer.eos_token_id

# Streaming setup
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

generation_kwargs = dict(
    **inputs,
    max_new_tokens=50,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    eos_token_id=eos_token_id,
    streamer=streamer,
)

# Start generation in parallel thread
thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

# Read tokens in real time
for new_text in streamer:
    print(new_text, end="", flush=True)

The Eiffel Tower is located in Paris, France. It is situated in the Champ de Mars, a public park located along the west bank of the Seine River. The Eiffel Tower is a popular tourist destination and is one of the